In [64]:
import folium
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from math import ceil

df_arbres = pd.read_csv("data/p2-arbres-fr.csv", sep=";",)
df_arbres.keys()
df_arbres.head()

,id,type_emplacement,domanialite,arrondissement,complement_addresse,numero,lieu,id_emplacement,libelle_francais,genre,espece,variete,circonference_cm,hauteur_m,stade_developpement,remarquable,geo_point_2d_a,geo_point_2d_b
0,99874,Arbre,Jardin,PARIS 7E ARRDT,NaN,NaN,MAIRIE DU 7E 116 RUE DE GRENELLE PARIS 7E,19,Marronnier,Aesculus,hippocastanum,NaN,20,5,NaN,0.0,48.857620,2.320962
1,99875,Arbre,Jardin,PARIS 7E ARRDT,NaN,NaN,MAIRIE DU 7E 116 RUE DE GRENELLE PARIS 7E,20,If,Taxus,baccata,NaN,65,8,A,NaN,48.857656,2.321031
2,99876,Arbre,Jardin,PARIS 7E ARRDT,NaN,NaN,MAIRIE DU 7E 116 RUE DE GRENELLE PARIS 7E,21,If,Taxus,baccata,NaN,90,10,A,NaN,48.857705,2.321061
3,99877,Arbre,Jardin,PARIS 7E ARRDT,NaN,NaN,MAIRIE DU 7E 116 RUE DE GRENELLE PARIS 7E,22,Erable,Acer,negundo,NaN,60,8,A,NaN,48.857722,2.321006
4,99878,Arbre,Jardin,PARIS 17E ARRDT,NaN,NaN,PARC CLICHY-BATIGNOLLES-MARTIN LUTHER KING,000G0037,Arbre à miel,Tetradium,daniellii,NaN,38,0,NaN,NaN,48.890435,2.315289


Creation carte des arbres les plus imposants

In [65]:

df_arbres[["circonference_cm","hauteur_m"]]
df_arbres = df_arbres.sort_values(by=["hauteur_m"])
# df_arbres = [row["hauteur_m"] if row["hauteur_m"] > 0 else " " for index, row in df_arbres.iterrows()]

# for index, row in df_arbres.iterrows():
#     if row["hauteur_m"] == 0:
#         df_arbres.drop(df_arbres.index[-2])
print(df_arbres["hauteur_m"])
len_df = len(df_arbres)
print("len",len_df)
mediane_rang = (len_df+1)/2
print("mediane_rang",int(round(mediane_rang)))
if int(mediane_rang) % 2 == 0:
    mediane =df_arbres.iloc[mediane_rang]
else:
    mediane = int((df_arbres["hauteur_m"].iloc[round(mediane_rang)]+df_arbres["hauteur_m"].iloc[round(mediane_rang)+1]))/2
print("mediane",mediane)

quartile_1 = df_arbres["hauteur_m"].iloc[ceil(len_df/4)]
quartile_3 = df_arbres["hauteur_m"].iloc[ceil(len_df*(3/4))]

print("quartile_1",quartile_1)
print("quartile_3",quartile_3)

200136         0
133704         0
133695         0
133692         0
133683         0
           ...  
152205      1815
101985      2015
152110      2525
150101      5155
182852    881818
Name: hauteur_m, Length: 200137, dtype: int64
len 200137
mediane_rang 100069
mediane 8.0
quartile_1 5
quartile_3 12


Supression des valeurs abbérentes

In [66]:
# for index, row in df_arbres.iterrows():
#     if row["hauteur_m"] == 0:
df_arbres = df_arbres.sort_values(by=["hauteur_m"])
df_arbres_sup_0 = df_arbres[df_arbres.hauteur_m != 0]
        # df_arbres = df_arbres.drop(df_arbres.index[-2])
# print(df_arbres_sup_0["hauteur_m"])
len_df = len(df_arbres_sup_0)
print("len",len_df)
mediane_rang = (len_df+1)/2
print("mediane_rang",int(round(mediane_rang)))
if int(mediane_rang) % 2 == 0:
    mediane =df_arbres_sup_0["hauteur_m"].iloc[int(mediane_rang)]
else:
    mediane = int((df_arbres_sup_0["hauteur_m"].iloc[round(mediane_rang)]+df_arbres_sup_0["hauteur_m"].iloc[round(mediane_rang)+1]))/2
print("mediane",mediane)

quartile_1 = df_arbres_sup_0["hauteur_m"].iloc[ceil(len_df/4)]
quartile_3 = df_arbres_sup_0["hauteur_m"].iloc[ceil(len_df*(3/4))]
ecart_inter_quartile = quartile_3 - quartile_1
valeur_aberrante_sup = quartile_3 + (1.5 * ecart_inter_quartile)
### On supprime les valeurs abberrantes
print("valeur_aberrante_sup",valeur_aberrante_sup)
df_arbres_sans_abberrante = df_arbres_sup_0[df_arbres_sup_0.hauteur_m < valeur_aberrante_sup]
print("df_arbres_sans_abberrante",df_arbres_sans_abberrante["hauteur_m"])
print("quartile_1",quartile_1)
print("quartile_3",quartile_3)
print("len",len(df_arbres_sans_abberrante))


len 160918
mediane_rang 80460
mediane 10.0
valeur_aberrante_sup 26.0
df_arbres_sans_abberrante 198071     1
197935     1
137084     1
197928     1
193654     1
          ..
121684    25
10549     25
121703    25
121680    25
44601     25
Name: hauteur_m, Length: 159526, dtype: int64
quartile_1 6
quartile_3 14
len 159526


In [68]:
df_arbres_sans_abberrante = df_arbres_sans_abberrante.sort_values(by=["hauteur_m"],ascending=False)
m = folium.Map(location=[48.85, 2.320], zoom_start=12, tiles="Stamen Terrain")
tooltip = "Clique pour avoir des infos sur l'arbre"
for index, row in df_arbres_sans_abberrante.head(100).iterrows():
    folium.Marker(
        [row["geo_point_2d_a"], row["geo_point_2d_b"]], popup=row["hauteur_m"]
        , tooltip=tooltip
    ).add_to(m)
m